In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import streamlit as st
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
import plotly.graph_objects as go
from sklearn.mixture import GaussianMixture
import plotly.express as px

from clusterbuster import parse_report, plot_clusters, gtype_gmm, calculate_maf, plot_hist_contour

Some info on the values:

R values are the intensities. R for a sample is the sum of normalized X and Y (R= X+ Y)
Theta is the B allele frequency and is calculated by: (2/pi)*arctan(normalized Y/ normalized X)

BAF = Y / (X + Y)
LRR = log2( (X + Y)sampleOfInterest / (X+Y)baselineSample)

In [2]:
# first, need to subset the huge report to just snps of interest
# report = pd.read_csv('F4SEttehadieh_P1_Genotyping_Report.txt', engine='c', sep='\t', dtype={'Chr':str, 'position':int})
# split by chromosome
# for chrom in report.Chr.unique():
#     report[report.Chr==chrom].to_csv(f'data/chr{chrom}_report.csv', header=True, index=False)

# report[report['GenTrain Score']<0.15]
# snps_df = pd.read_csv('prototyping/snps_of_interest_first_data.txt', engine='c')
# report[report.Name.isin(snps_df.Name)].to_csv('data/cb_test_snps.txt', sep='\t', index=False)

# check_header = pd.read_csv('../data/chr22_report.csv', nrows=0)
# header_list = ['Index', 'Name', 'Address', 'Chr', 'Position', 'GenTrain Score','Frac A', 'Frac C', 'Frac G', 'Frac T']
# all(elem in check_header.columns for elem in header_list)
# check_header.columns


In [3]:
report_in = pd.read_csv('../data/chr22_report.csv', engine='c', dtype={'Chr':str, 'position':int})

report = parse_report(report_in, flag_maf=0.01, flag_gentrain=0.5)
# test_report.keys()
cb_df = report['clusterbuster_df']
snps_df = report['flagged_snps']
flagged = snps_df[snps_df.maf_flag & snps_df.gentrain_flag]

total_flagged = snps_df[snps_df.maf_flag | snps_df.gentrain_flag].reset_index()
both_flagged = snps_df[snps_df.maf_flag & snps_df.gentrain_flag].reset_index()
maf_flagged = snps_df.loc[snps_df.maf_flag].reset_index()
gentrain_flagged = snps_df.loc[snps_df.gentrain_flag].reset_index()

In [4]:
# selected from user input
selected_snp = 'exm1601793'
# selected_snp = 'JHU_22.27408320'

geno_col = selected_snp + ".GType"
theta_col = selected_snp + ".Theta"
r_col = selected_snp + ".R"

gtypes = list(cb_df[geno_col].unique())

snp_for_plot_df = cb_df[['IID', theta_col, r_col, geno_col]].copy()
snp_for_plot_df.columns = snp_for_plot_df.columns.str.replace(f'{selected_snp}.', '', regex=False)

df = snp_for_plot_df.copy()
x_col, y_col = 'Theta', 'R'
gtypes_list = list(df.GType.unique())

df = snp_for_plot_df.copy()
x_col, y_col, gtype_col = 'Theta', 'R', 'GType'
gtypes_list = (df.GType.unique())

clusters_fig = plot_clusters(df=df, x_col=x_col, y_col=y_col, gtype_col=gtype_col, snpid=selected_snp)


# clusters_fig.show()

In [5]:
# now train a gmm and plot reclustered variants
# get genotypes and number for choosing number of components
gtypes_for_gmm = list(cb_df[geno_col].unique())
gtypes_for_gmm.remove('NC')
n_components = len(gtypes_for_gmm)

snp_for_gmm_df = cb_df[['IID', theta_col, r_col, geno_col]].copy()
snp_for_gmm_df.columns = snp_for_gmm_df.columns.str.replace(f'{selected_snp}.','', regex=False)
gmm_out = gtype_gmm(snp_theta_r_df=snp_for_gmm_df, n_components=n_components)

gmm_plot_df = gmm_out['X']
gmm_plot_df.loc[:,'GType'] = gmm_out['y_pred']
gmm_plot_df.loc[:,'IID'] = gmm_out['IID']
gmm = gmm_out['gmm']

cluster_means = {gtype:gmm_plot_df.loc[gmm_plot_df.GType==gtype,'Theta'].mean() for gtype in gmm_plot_df.GType.unique()}

gtype_order = ['AA','AB','BB']
gtypes_for_gmm_ordered = [gtype for gtype in gtype_order if gtype in gtypes_for_gmm]
ordered_cluster_means = dict(sorted(cluster_means.items(), key=lambda item: item[1]))
gmm_gtype_map = {list(ordered_cluster_means.keys())[i]:gtype for i, gtype in enumerate(gtypes_for_gmm_ordered)}

gmm_plot_df.loc[:,'gtype_out'] = gmm_plot_df.loc[:,'GType'].replace(gmm_gtype_map)
gmm_plot_df.loc[:,'original_gtype'] = snp_for_gmm_df.loc[:,'GType']
gmm_plot_df
# recluster_fig = plot_hist_contour(gmm_plot_df, x_col='Theta', y_col='R', gtype_col='gtype_out')

TypeError: plot_hist_contour() missing 2 required positional arguments: 'xlim' and 'ylim'

In [6]:
# fig = go.make_subplots(
#     rows=1, cols=2,shared_xaxes=True, specs=[[{'type': 'scatterplot'}, {'type': 'polar'}],
#                            [{'type': 'scene'}, {'type': 'ternary'}]])

In [73]:
d3 = px.colors.qualitative.D3

cmap = {
    'AA': d3[0],
    'AB': d3[1],
    'BA': d3[1],
    'BB': d3[2],
    'NC': d3[3]
}

cluster_fig = px.scatter(
        df, 
        x=x_col, y=y_col, 
        color=gtype_col, 
        color_discrete_map=cmap, 
        width=600, height=400)
cluster_fig.show()

In [74]:






n_trace = len(gmm_plot_df['gtype_out'].unique())


recluster_fig = px.density_contour(
    gmm_plot_df,
    x=x_col, y=y_col, 
    marginal_x="histogram", 
    marginal_y="histogram",
    color="gtype_out", 
    color_discrete_map=cmap
    )

fig2 = px.scatter(gmm_plot_df, x=x_col, y=y_col, color='gtype_out', color_discrete_map=cmap)

# fig2.update_xaxes(range=xlim, nticks=10)
# fig2.update_yaxes(range=ylim, nticks=10)


for i in range(n_trace):
    recluster_fig.add_trace(fig2.data[i])
recluster_fig.show()

In [75]:
import plotly.subplots as sp
figure1_traces = []
figure2_traces = []
for trace in range(len(cluster_fig["data"])):
    figure1_traces.append(cluster_fig["data"][trace])
for trace in range(len(fig2["data"])):
    figure2_traces.append(fig2["data"][trace])

#Create a 1x2 subplot
out_fig = sp.make_subplots(rows=1, cols=2) 

# Get the Express fig broken down as traces and add the traces to the proper plot within in the subplot
for traces in figure1_traces:
    out_fig.append_trace(traces, row=1, col=1)
for traces in figure2_traces:
    out_fig.append_trace(traces, row=1, col=2)

In [76]:
# for trace in range(len(this_figure.data)):
#     print(this_figure[0])
for gtype in gmm_plot_df['gtype_out'].unique():
    x = gmm_plot_df.loc[gmm_plot_df['gtype_out']==gtype, x_col]
    y = gmm_plot_df.loc[gmm_plot_df['gtype_out']==gtype, y_col]
    out_fig.append_trace(go.Histogram2dContour(
        x = x,
        y = y,
        xaxis = 'x',
        yaxis = 'y',
    ), 1,2)

    # out_fig.append_trace(go.Histogram(
    #     y = y,
    #     xaxis = 'x2',
    #     ),
    #     1,2)
    # out_fig.append_trace(go.Histogram(
    #     x = x,
    #     yaxis = 'y2',
    #     ),
    #     1,2)
    
    # out_fig.update_layout(
    #     autosize = False,
    #     xaxis = dict(
    #         zeroline = False,
    #         domain = [0,0.85],
    #         showgrid = False
    #     ),
    #     yaxis = dict(
    #         zeroline = False,
    #         domain = [0,0.85],
    #         showgrid = False
    #     ),
    #     xaxis2 = dict(
    #         zeroline = False,
    #         domain = [0.85,1],
    #         showgrid = False
    #     ),
    #     yaxis2 = dict(
    #         zeroline = False,
    #         domain = [0.85,1],
    #         showgrid = False
    #     ),
    #     height = 600,
    #     width = 600,
    #     bargap = 0,
    #     hovermode = 'closest',
    #     showlegend = False
    # )


In [77]:
out_fig.show()

In [13]:
def plot_hist_contour(df, x_col, y_col, gtype_col, xlim, ylim):

    d3 = px.colors.qualitative.D3

    cmap = {
        'AA': d3[0],
        'AB': d3[1],
        'BA': d3[1],
        'BB': d3[2],
        'NC': d3[3]
    }

    n_trace = len(df[gtype_col].unique())


    fig = px.density_contour(
        df,
        x=x_col, y=y_col, 
        marginal_x="histogram", 
        marginal_y="histogram",
        color="gtype_out", 
        color_discrete_map=cmap,
        width=750, height=500
        )


    fig2 = px.scatter(df, x=x_col, y=y_col, color=gtype_col, color_discrete_map=cmap)
    fig2.update_xaxes(range=xlim, nticks=10)
    fig2.update_yaxes(range=ylim, nticks=10)
   

    for i in range(n_trace):
        fig.add_trace(fig2.data[i])

    return fig


def plot_clusters(df, x_col, y_col, gtype_col, snpid):

    d3 = px.colors.qualitative.D3

    cmap = {
        'AA': d3[0],
        'AB': d3[1],
        'BA': d3[1],
        'BB': d3[2],
        'NC': d3[3]
    }

    # gtypes_list = (df[gtype_col].unique())
    xmin, xmax = df[x_col].min(), df[x_col].max()
    ymin, ymax = df[y_col].min(), df[y_col].max()

    xlim = [xmin-.1, xmax+.1]
    ylim = [ymin-.1, ymax+.1]

    fig = px.scatter(
        df, 
        x=x_col, y=y_col, 
        color=gtype_col, 
        color_discrete_map=cmap, 
        width=600, height=400)

    fig.update_xaxes(range=xlim, nticks=10)
    fig.update_yaxes(range=ylim, nticks=10)

    out_dict = {
        'fig': fig,
        'xlim': xlim,
        'ylim': ylim
    }

    return out_dict

In [88]:
sample_ids = ['a','b','c']
gene_list = ['snca','snca','snca']
cnv_list = ['1','2','3']
sample_id = 'b'
gene = 'snca'


cnv_table = pd.DataFrame(
    {
        'sample_id':sample_ids,
        'gene':gene_list,
        'cnv_type':cnv_list
        }
    )

cnv_table[(cnv_table.sample_id==sample_id) & (cnv_table.gene==gene)].reset_index().cnv_type[0]

'2'

In [204]:
# d3 = px.colors.qualitative.D3
# cmap = {
#     'AA': d3[0],
#     'AB': d3[1],
#     'BA': d3[1],
#     'BB': d3[2],
#     'NC': d3[3]
# }

# fig = px.density_contour(gmm_plot_df, x='Theta', y='R', marginal_x="histogram", marginal_y="histogram",color="gtype_out", color_discrete_map=cmap, opacity=.8)
# fig2 = px.scatter(gmm_plot_df, x='Theta', y='R', color="gtype_out", color_discrete_map=cmap)

# fig.add_trace(fig2.data[0]), fig.add_trace(fig2.data[1]), fig.add_trace(fig2.data[2])
# fig.data[0].colorscale='blues'
# fig.data[0].contours['coloring']='fill'
# fig.data[3].contours['coloring']='fill'
# fig.data[6].contours['coloring']='fill'

# fig.data[0].showlegend=False
# fig.data[3].showlegend=False
# fig.data[6].showlegend=False
gmm_plot_df

idx,Theta,R,GType,IID,gtype_out,original_gtype
0,0.978341,1.124041,1,FS04821313_FO0000275_FUS_R495*_A01_17_A01,BB,BB
1,0.950462,1.286430,1,FS04821314_FO0000152_MAPT_R317W_A08_17_B01,BB,BB
2,0.974167,1.143932,1,FS04821315_FO0000163_MAPT_R317W_G09_17_B12,BB,BB
3,0.959375,1.239037,1,FS04821316_FO0000176_ANG_K41I_A04_17_D01,BB,BB
4,0.954499,1.171495,1,FS04821317_FO0000188_SNCB_P123H_B08_17_E01,BB,BB
...,...,...,...,...,...,...
91,0.958348,1.160252,1,FS04821404_FO0000171_APP_A673T_D09_17_C08,BB,BB
92,0.952213,1.151133,1,FS04821405_FO0000172_APP_A673T_D12_17_C09,BB,BB
93,0.013725,1.494911,2,FS04821406_FO0000173_APP_A673T_F06_17_C10,AA,AA
94,0.964469,1.133834,1,FS04821407_FO0000174_APP_A673T_H08_17_C11,BB,BB


In [91]:
import pandas as pd
h5 = '/home/dan_vitale/projects/genotools_streamlit/GP2_round1.QC.metrics.h5'

df_qc = pd.read_hdf(h5, key='QC')
df_ancestry_counts = pd.read_hdf(h5, key='ancestry_counts')
df_ancestry_labels = pd.read_hdf(h5, key='ancestry_labels')
df_confusion_matrix = pd.read_hdf(h5, key='confusion_matrix')
df_new_samples_umap = pd.read_hdf(h5, key='new_samples_umap')
df_projected_pcs = pd.read_hdf(h5, key='projected_pcs')
df_ref_pcs = pd.read_hdf(h5, key='ref_pcs')
df_ref_umap = pd.read_hdf(h5, key='ref_umap')
df_total_umap = pd.read_hdf(h5, key='total_umap')





In [92]:

df_qc.to_csv('data/df_qc.csv')
df_ancestry_counts.to_csv('data/df_ancestry_counts.csv', index=False)
df_ancestry_labels.to_csv('data/df_ancestry_labels.csv', index=False)
df_projected_pcs.to_csv('data/df_projected_pcs.csv',index=False)
df_ref_pcs.to_csv('data/df_ref_pcs.csv', index=False)



In [94]:
#  gwas = pd.read_csv('https://raw.githubusercontent.com/plotly/dash-bio-docs-files/master/manhattan_data.csv')
test = ['a']
test.append('b')
test

['a', 'b']

In [215]:
import plotly.graph_objects as go

import numpy as np

t = np.linspace(-1, 1.2, 2000)
# x = (t**3) + (0.3 * np.random.randn(2000))
# y = (t**6) + (0.3 * np.random.randn(2000))




d3 = px.colors.qualitative.D3

fig = go.Figure()

cmap = {
    'AA': d3[0],
    'AB': d3[1],
    'BA': d3[1],
    'BB': d3[2],
    'NC': d3[3]
}



for i, gtype in enumerate(gtypes):
    print(i, gtype)

    x = gmm_plot_df.loc[gmm_plot_df.GType==gtype,'Theta']
    y = gmm_plot_df.loc[gmm_plot_df.GType==gtype,'R']

    fig.add_trace(go.Histogram2dContour(
        x = x,
        y = y,
        colorscale = 'Blues',
        reversescale = True,
        xaxis = f'x',
        yaxis = f'y'
    ))

    fig.add_trace(go.Scatter(
            x = x,
            y = y,
            xaxis = f'x',
            yaxis = f'y',
            mode = 'markers',
            marker = dict(
                color = cmap[gtype],
                size = 3
            )
        ))

    fig.add_trace(go.Histogram(
            y = y,
            xaxis = f'x2',
            marker = dict(
                color = cmap[gtype]
            )
        ))

    fig.add_trace(go.Histogram(
            x = x,
            yaxis = f'y2',
            marker = dict(
                color = cmap[gtype]
            )
        ))

    fig.update_layout(
        autosize = False,
        xaxis = dict(
            zeroline = False,
            domain = [0,0.85],
            showgrid = False
        ),
        yaxis = dict(
            zeroline = False,
            domain = [0,0.85],
            showgrid = False
        ),
        xaxis2 = dict(
            zeroline = False,
            domain = [0.85,1],
            showgrid = False
        ),
        yaxis2 = dict(
            zeroline = False,
            domain = [0.85,1],
            showgrid = False
        ),
        height = 600,
        width = 600,
        bargap = 0,
        hovermode = 'closest',
        showlegend = False
    )

fig.show()

0 BB
1 AA
2 AB


In [85]:
def plot_gmm(df, x_col, y_col, gtype_col, gmm, snpid, n_std=3):

    d3 = px.colors.qualitative.D3

    cmap = {
        'AA': d3[0],
        'AB': d3[1],
        'BA': d3[1],
        'BB': d3[2],
        'NC': d3[3]
    }

    gtypes_list = (df[gtype_col].unique())

    fig = go.Figure()

    for gtype in gtypes_list:
        df_ = df.loc[df[gtype_col]==gtype]
        x = df_.loc[:, x_col]
        y = df_.loc[:, y_col]
        color = cmap[gtype]
        fig.add_trace(
            go.Scatter(
                x=x, y=y, 
                mode="markers",
                name=gtype,
                marker = {'color':color},
                hovertemplate="Genotype=%s<br>Theta=%%{x}<br>R=%%{y}<extra></extra>"% gtype
                ))

        # fig.update_layout(title_text=f'{snpid} Cluster Plot')
        # fig.update_layout(legend_title_text = 'Genotype')
        # fig.update_xaxes(title_text=x_col)
        # fig.update_yaxes(title_text=y_col, tickangle=-90)

        # fig.add_trace(go.Histogram2dContour(
        # x = x,
        # y = y,
        # showlegend=False
        # ))

        
        # for n_std in range(1, n_std):
        #     fig.add_shape(type='path',
        #     path=confidence_ellipse(x, y, n_std=1.96*n_std, size=100), 
        #     line_color=color,
        #     fillcolor=color,
        #     opacity=0.2)
    
    return fig

In [86]:
plot_gmm(df=gmm_plot_df, x_col='Theta', y_col='R', gtype_col='gtype_out', gmm=gmm, snpid=selected_snp, n_std=5)

In [48]:
import plotly.figure_factory as ff
import numpy as np

# Add histogram data
x1 = np.random.randn(200)-2
x2 = np.random.randn(200)
x3 = np.random.randn(200)+2
x4 = np.random.randn(200)+4

# Group data together
hist_data = [x1, x2, x3, x4]

group_labels = ['Group 1', 'Group 2', 'Group 3', 'Group 4']

# Create distplot with custom bin_size
fig = ff.create_distplot(hist_data, group_labels, bin_size=[.1, .25, .5, 1])
fig.show()

In [49]:
cb_df

idx,IID,1KG_22_19435270.GType,1KG_22_19766749.GType,1KG_22_24530438.GType,1KG_22_24909316.GType,1KG_22_26286764.GType,1KG_22_26298671.GType,1KG_22_30857665.GType,1KG_22_36681998.GType,1KG_22_38494134.GType,...,Variant61228.R,Variant61231.R,Variant61234.R,Variant61237.R,Variant61242.R,Variant61244.R,Variant61245.R,Variant61246.R,Variant61252.R,Variant61253.R
0,FS04821313_FO0000275_FUS_R495*_A01_17_A01,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.430500,0.361874,0.540405,0.626420,0.957749,0.733668,0.795464,0.701761,0.470309,0.569939
1,FS04821314_FO0000152_MAPT_R317W_A08_17_B01,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.415114,0.329952,0.549609,0.571324,0.932040,0.671134,0.730768,0.735666,0.476564,0.603078
2,FS04821315_FO0000163_MAPT_R317W_G09_17_B12,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.379468,0.420835,0.541583,0.608452,0.976639,0.641460,0.726527,0.748987,0.439781,0.518233
3,FS04821316_FO0000176_ANG_K41I_A04_17_D01,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.462127,0.387539,0.521502,0.639255,0.965216,0.699769,0.735392,0.791597,0.410290,0.626747
4,FS04821317_FO0000188_SNCB_P123H_B08_17_E01,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.469232,0.366763,0.586054,0.579106,1.041958,0.677856,0.758849,0.712157,0.468440,0.582567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,FS04821404_FO0000171_APP_A673T_D09_17_C08,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.359078,0.360436,0.449827,0.581284,1.042775,0.702279,0.757134,0.718126,0.447970,0.575152
92,FS04821405_FO0000172_APP_A673T_D12_17_C09,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.365155,0.359822,0.513683,0.557322,0.872211,0.649222,0.692903,0.726661,0.427236,0.530507
93,FS04821406_FO0000173_APP_A673T_F06_17_C10,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.329993,0.388541,0.518266,0.577355,0.891683,0.711396,0.780114,0.710056,0.407510,0.513913
94,FS04821407_FO0000174_APP_A673T_H08_17_C11,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.354541,0.322347,0.421428,0.515936,0.894079,0.712438,0.747326,0.663459,0.403457,0.479983


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.mixture import GaussianMixture

gtypes_map = {
    'AA':'0',
    'AB':'1',
    'BA':'1',
    'BB':'2',
    }

gtypes_map_rev = {
    '0':'AA',
    '1':'AB',
    '2':'BB'
}

# samples to train and test on
to_gmm_samples = snp_for_gmm_df.loc[snp_for_gmm_df[geno_col]!='NC']
# to_gmm_samples.loc[:,'gtype'] = np.where(to_gmm_samples.loc[:, geno_col]=='BA', 'AB', to_gmm_samples.loc[:, geno_col])

# samples to predict
to_pred_samples = snp_for_gmm_df.loc[snp_for_gmm_df[geno_col]=='NC']

X = to_gmm_samples.loc[:,[theta_col, r_col]]
y = np.where(to_gmm_samples.loc[:, geno_col]=='BA', 'AB', to_gmm_samples.loc[:, geno_col])
y = to_gmm_samples.loc[:,geno_col].replace(gtypes_map)

x_pred = to_pred_samples.loc[:,[theta_col, r_col]]
# n_classes = len(np.unique(y))
classes = np.unique(y)

gmm = GaussianMixture(
        n_components=n_components,
        covariance_type="diag",
        random_state = 10).fit(X)

gmm.means_ = np.array([X[y == i].mean(axis=0) for i in classes])

gmm.fit(X)
y_pred = gmm.predict(x_pred)
y_pred

In [36]:
# import math
# position = gmm.means_
# covariances = gmm.covariances_
# w = gmm.weights_

# position[0][0]

# def ellipse(x_center=0, y_center=0, ax1 = [1, 0],  ax2 = [0,1], a=1, b =1,  N=100):
#    # x_center, y_center the coordinates of ellipse center
#    # ax1 ax2 two orthonormal vectors representing the ellipse axis directions
#    # a, b the ellipse parameters
#    if np.linalg.norm(ax1) != 1 or np.linalg.norm(ax2) != 1:
#        raise ValueError('ax1, ax2 must be unit vectors')
#    if  abs(np.dot(ax1, ax2)) > 1e-06:
#        raise ValueError('ax1, ax2 must be orthogonal vectors')
#    #rotation matrix   
#    R = np.array([ax1, ax2]).T
#    if np.linalg.det(R) <0: 
#        raise ValueError("the det(R) must be positive to get a  positively oriented ellipse reference frame")
#    t = np.linspace(0, 2*math.pi, N)
#    #ellipse parameterization with respect to a system of axes of directions a1, a2
#    xs = a * math.cos(t)
#    ys = b * math.sin(t)
   
#    # coordinate of the  ellipse points with respect to the system of axes [1, 0], [0,1] with origin (0,0)
#    xp, yp = np.dot(R, [xs, ys])
#    x = xp + x_center 
#    y = yp + y_center
#    return x, y


# ellipse(x_center=position[0], y_center=position[1])

0.896737899999998

In [11]:
# snp_list = gtype_df.idx
# snps_of_interest = list()
# count=0
# for snp in snp_list:
#     if set(['NC','AA','BB']).issubset(list(GType_transposed_df[snp].unique())):
#         vc = GType_transposed_df[snp].value_counts()
#         # print(vc)
#         snps_of_interest.append(snp)
#         count+=1
       
# print(count)

39


In [13]:
# # flag_df[(flag_df.maf>0.03) & (flag_df.maf<0.5)
# snps_of_interest_df = pd.DataFrame({'snp':snps_of_interest})
# snps_of_interest_df.loc[:,'snp'] = snps_of_interest_df.loc[:,'snp'].str.replace('.GType','',regex=False) 
# snps_of_interest_df.to_csv('../data/testing_snps_of_interest.csv', index=False)

In [26]:
# Chop out to GType, Score and Theta dataframes.
flag_maf=0.01
flag_gentrain=0.5
report = report_in.copy()
Name_df = report[['Name']].copy()
GType_df = report.loc[:, report.columns.str.endswith(".GType")]
Theta_df = report.loc[:, report.columns.str.endswith(".Theta")]
R_df = report.loc[:, report.columns.str.endswith(".R")]

Name_df['Name.GType'] = Name_df.loc[:,'Name'] + ".GType"
Name_df['Name.Theta'] = Name_df.loc[:,'Name'] + ".Theta"
Name_df['Name.R'] = Name_df.loc[:,'Name'] + ".R"

# Merge names plus data types.
Name_GType_df = pd.concat([Name_df, GType_df], axis=1).rename(columns={"Name.GType":"idx"}).drop(columns=['Name', 'Name.Theta', 'Name.R'])
Name_GType_df.columns = Name_GType_df.columns.str.replace(".GType", "", regex=False)
Name_GType_df_final = Name_GType_df.set_index('idx')

Name_Theta_df = pd.concat([Name_df, Theta_df], axis=1).rename(columns={"Name.Theta": "idx"}).set_index('idx').drop(columns=['Name', 'Name.GType', 'Name.R'])
Name_Theta_df.columns = Name_Theta_df.columns.str.replace(".Theta", "", regex=False)

Name_R_df = pd.concat([Name_df, R_df], axis=1).rename(columns={"Name.R":"idx"}).set_index('idx').drop(columns=['Name', 'Name.Theta', 'Name.GType'])
Name_R_df.columns = Name_R_df.columns.str.replace(".R", "", regex=False)

#  Transpose the data frames and make the names of the variants plus the suffixes the columns.
GType_transposed_df = Name_GType_df_final.transpose()
Theta_transposed_df = Name_Theta_df.transpose()
R_transposed_df = Name_R_df.transpose()

# Smash everything together and get ready for plotting.
temp_df = GType_transposed_df.merge(Theta_transposed_df, left_index=True, right_index=True)
clusterbuster_df = temp_df.merge(R_transposed_df, left_index=True, right_index=True)
clusterbuster_out = clusterbuster_df.reset_index().rename(columns={'index':'IID'})

# get gentrain scores

gtrain_scores_df = report_in.loc[:,['Name', 'Chr', 'Position','GenTrain Score','Frac A', 'Frac C', 'Frac G', 'Frac T']]
gtrain_scores_df.columns = ['snpid','Chr','Position','gentrain_score','Frac_A','Frac_C','Frac_G','Frac_T']
# calculate maf
gtype_df = Name_GType_df.copy()
gtype_df.loc[:,'snpid'] = gtype_df.loc[:,'idx'].str.replace(".GType", "", regex=False)
gtype_to_maf = gtype_df.drop(columns=['idx'])
maf_scores_df = calculate_maf(gtype_to_maf)
flag_df = maf_scores_df.merge(gtrain_scores_df, how='inner', on='snpid')
flag_df.loc[:,'maf_flag'] = np.where(flag_df.maf<flag_maf, True, False)
flag_df.loc[:,'gentrain_flag'] = np.where(flag_df.gentrain_score<flag_gentrain, True, False)

missing_df = pd.DataFrame()


out_dict = {
    'clusterbuster_df': clusterbuster_out,
    'flagged_snps': flag_df, 
}

In [36]:
# Name_Theta_df[Name_Theta_df.isna(axis=1)]

Name_Theta_df[Name_Theta_df.isnull().any(axis=1)]

,FS04821313_FO0000275_FUS_R495*_A01_17_A01,FS04821314_FO0000152_MAPT_R317W_A08_17_B01,FS04821315_FO0000163_MAPT_R317W_G09_17_B12,FS04821316_FO0000176_ANG_K41I_A04_17_D01,FS04821317_FO0000188_SNCB_P123H_B08_17_E01,FS04821318_FO0000200_PFN1_C71G_A01_17_F01,FS04821319_FO0000212_HNRNPA1_D262N_A01_17_G01,FS04821320_FO0000224_CSF1R_E633K_A01_17_H01,FS04821321_FO0000276_FUS_R495*_A04_17_A02,FS04821322_FO0000153_MAPT_R317W_A10_17_B02,...,FS04821399_FO0000166_APP_A673T_A10_17_C03,FS04821400_FO0000167_APP_A673T_B02_17_C04,FS04821401_FO0000168_APP_A673T_B11_17_C05,FS04821402_FO0000169_APP_A673T_C03_17_C06,FS04821403_FO0000170_APP_A673T_C04_17_C07,FS04821404_FO0000171_APP_A673T_D09_17_C08,FS04821405_FO0000172_APP_A673T_D12_17_C09,FS04821406_FO0000173_APP_A673T_F06_17_C10,FS04821407_FO0000174_APP_A673T_H08_17_C11,PositiveControl_1_NA_NA
idx,,,,,,,,,,,,,,,,,,,,,
22:19263188-TC.Theta,0.016549,0.016549,0.034746,0.005748,0.013034,0.028882,0.036758,0.009955,0.027756,0.026663,...,0.017424,0.030915,0.026956,0.011431,0.021527,0.011655,0.035450,0.039059,0.029760,0.050559
22:21354461-GA.Theta,0.958369,0.881275,0.842105,NaN,0.900391,0.901443,0.901834,0.950166,0.885677,0.955232,...,0.856874,0.896709,0.903836,0.946666,0.937902,0.927664,0.946430,0.931566,0.867493,0.524616
22:21384250-AG.Theta,0.040352,0.093047,0.065083,0.079220,0.063334,0.056559,0.088066,0.097245,0.146142,0.121591,...,0.101050,0.109888,0.090588,0.062786,0.094529,0.121504,0.117310,0.072148,0.136329,0.434891
22:21984279-CA.Theta,0.936341,0.912554,0.924742,0.921270,0.957253,0.954720,0.900188,0.946820,0.911227,0.937269,...,0.880905,0.919169,0.942667,0.971814,0.899858,0.977350,0.914369,0.905627,0.905158,0.607808
22:24481101-AG.Theta,0.027583,0.029278,0.020352,0.033576,0.017459,0.029715,0.028005,0.025697,0.014463,0.022326,...,0.030915,0.023189,0.010187,0.016286,0.017335,0.018646,0.020970,0.031749,0.016931,0.029535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
rs9625374.Theta,0.578136,0.577489,0.562329,0.567274,0.587530,0.578070,0.591982,0.696036,0.603432,0.613582,...,0.618050,0.568534,0.558752,0.603907,0.591012,0.579917,0.589297,0.626615,0.603600,0.019538
rs9626059.Theta,0.959312,0.956198,0.945253,0.927606,0.948385,0.953619,0.958424,0.931868,0.926692,0.931908,...,0.911472,0.917925,0.952390,0.973589,0.954016,0.950417,0.937735,0.960297,0.951617,0.920231
rs9628047.Theta,0.515638,0.536284,0.560650,0.539354,0.543270,0.538346,0.570527,0.482153,0.550294,0.578773,...,0.520268,0.537218,0.557735,0.506185,0.537496,0.575556,0.550700,0.554986,0.564009,0.518252


In [17]:
# Generate output report
for snp in snps_list:
    geno_col = snp + ".GType"
    theta_col = snp + ".Theta"
    r_col = snp + ".R"    

    gtypes = list(cb_df[geno_col].unique())

    snp_for_plot_df = cb_df[['IID', theta_col, r_col, geno_col]].copy()
    snp_for_plot_df.columns = snp_for_plot_df.columns.str.replace(f'{snp}.', '', regex=False)

NameError: name 'snps_list' is not defined

In [35]:
clusterbuster_out

idx,IID,1KG_22_19435270.GType,1KG_22_19766749.GType,1KG_22_24530438.GType,1KG_22_24909316.GType,1KG_22_26286764.GType,1KG_22_26298671.GType,1KG_22_30857665.GType,1KG_22_36681998.GType,1KG_22_38494134.GType,...,Variant61228.R,Variant61231.R,Variant61234.R,Variant61237.R,Variant61242.R,Variant61244.R,Variant61245.R,Variant61246.R,Variant61252.R,Variant61253.R
0,FS04821313_FO0000275_FUS_R495*_A01_17_A01,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.430500,0.361874,0.540405,0.626420,0.957749,0.733668,0.795464,0.701761,0.470309,0.569939
1,FS04821314_FO0000152_MAPT_R317W_A08_17_B01,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.415114,0.329952,0.549609,0.571324,0.932040,0.671134,0.730768,0.735666,0.476564,0.603078
2,FS04821315_FO0000163_MAPT_R317W_G09_17_B12,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.379468,0.420835,0.541583,0.608452,0.976639,0.641460,0.726527,0.748987,0.439781,0.518233
3,FS04821316_FO0000176_ANG_K41I_A04_17_D01,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.462127,0.387539,0.521502,0.639255,0.965216,0.699769,0.735392,0.791597,0.410290,0.626747
4,FS04821317_FO0000188_SNCB_P123H_B08_17_E01,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.469232,0.366763,0.586054,0.579106,1.041958,0.677856,0.758849,0.712157,0.468440,0.582567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,FS04821404_FO0000171_APP_A673T_D09_17_C08,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.359078,0.360436,0.449827,0.581284,1.042775,0.702279,0.757134,0.718126,0.447970,0.575152
92,FS04821405_FO0000172_APP_A673T_D12_17_C09,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.365155,0.359822,0.513683,0.557322,0.872211,0.649222,0.692903,0.726661,0.427236,0.530507
93,FS04821406_FO0000173_APP_A673T_F06_17_C10,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.329993,0.388541,0.518266,0.577355,0.891683,0.711396,0.780114,0.710056,0.407510,0.513913
94,FS04821407_FO0000174_APP_A673T_H08_17_C11,BB,BB,BB,BB,AA,AA,AA,AA,AA,...,0.354541,0.322347,0.421428,0.515936,0.894079,0.712438,0.747326,0.663459,0.403457,0.479983
